###1. 학습 준비

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# df_93이라는 데이터가 있다고 가정
df_93 = pd.read_csv("/content/drive/MyDrive/KorCCViD_v1.3_fullcleansed.csv")

In [ ]:
# 데이터를 학습 데이터와 테스트 데이터로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(df_93['Transcript'], df_93['Label'], test_size=0.2, random_state=42)

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 43.9 MB/s eta 0:00:00


In [ ]:
import os
from konlpy.tag import Okt

# Okt 객체 생성
okt = Okt()

def tokenize(texts, saved_filename):
    """주어진 텍스트 리스트를 토큰화하고 진행 상황을 출력합니다."""
    tokenized_texts = []

    # 이전에 저장된 토큰화 결과가 있다면 불러옵니다.
    if os.path.exists(saved_filename):
        with open(saved_filename, 'r', encoding='utf-8') as f:
            tokenized_texts = [line.strip() for line in f]

    start_idx = len(tokenized_texts)
    for idx, text in enumerate(texts[start_idx:]):
        if (idx + 1) % 200 == 0:
            print(f"{idx + 1 + start_idx}개의 텍스트를 토큰화했습니다.")
            # 중간 결과를 저장합니다.
            with open(saved_filename, 'a', encoding='utf-8') as f:
                for tokens in tokenized_texts[idx-199:idx+1]:
                    f.write(' '.join(tokens) + '\n')

        tokenized_texts.append(okt.morphs(text))

    return tokenized_texts

# 학습 데이터와 테스트 데이터를 토큰화합니다.
X_train_tokenized = tokenize(X_train, 'X_train_tokenized.txt')
X_test_tokenized = tokenize(X_test, 'X_test_tokenized.txt')

print("토큰화 완료!")


200개의 텍스트를 토큰화했습니다.
400개의 텍스트를 토큰화했습니다.
600개의 텍스트를 토큰화했습니다.
800개의 텍스트를 토큰화했습니다.
200개의 텍스트를 토큰화했습니다.
토큰화 완료!


In [ ]:
# 학습할 때 tokenizer의 상태 저장하기
import pickle
with open('/content/drive/MyDrive/finalvoice/bi_lstm_attention_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# 예측할 때 tokenizer의 상태 불러오기
with open('/content/drive/MyDrive/finalvoice/bi_lstm_attention_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

### 2 Attention + LSTM 모델


#### 2.1. 모델 설계


2.1.1. Attention 매커니즘

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Attention 가중치를 위한 weight 생성
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        # Attention score 계산
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


2.1.2. LSTM 모델 설계

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizer 객체 생성 및 학습
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_tokenized)

# 텍스트 데이터를 숫자로 변환
X_train_sequences = tokenizer.texts_to_sequences(X_train_tokenized)
X_test_sequences = tokenizer.texts_to_sequences(X_test_tokenized)

# 패딩 처리
MAX_LENGTH = max(len(s) for s in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=MAX_LENGTH)
X_test_padded = pad_sequences(X_test_sequences, maxlen=MAX_LENGTH)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Concatenate, BatchNormalization, Input

VOCAB_SIZE = len(tokenizer.word_index) + 1
EMBEDDING_DIM = 128

# 모델 구성
input_layer = Input(shape=(MAX_LENGTH,))
embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(input_layer)
bi_lstm = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention = AttentionLayer()(bi_lstm)
output = Dense(1, activation='sigmoid')(attention)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# 모델 학습
model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_data=(X_test_padded, y_test))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 128)          1126912   
                                                                 
 bidirectional (Bidirectiona  (None, 100, 128)         98816     
 l)                                                              
                                                                 
 attention_layer (AttentionL  (None, 128)              228       
 ayer)                                                           
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,226,085
Trainable params: 1,226,085
Non-train

In [ ]:
model.save("/content/drive/MyDrive/finalvoice/bi_lstm_attention_model.h5")

## 여기부터 돌리면됨

In [ ]:
!pip install konlpy
import os
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
okt = Okt()
# tokenizer = Tokenizer()
import pickle
# 예측할 때 tokenizer의 상태 불러오기
with open('/content/drive/MyDrive/finalvoice/bi_lstm_attention_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
import tensorflow as tf
from tensorflow.keras.layers import Layer
# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/finalvoice/bi_lstm_attention_model.h5", custom_objects={'AttentionLayer': AttentionLayer})

# 커스템 레이어 정의
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Attention 가중치를 위한 weight 생성
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        # Attention score 계산
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

def predict_label_prob(sentence):
    # 문장 토큰화
    tokenized_sentence = okt.morphs(sentence)
    print(f"토큰화된 문장: {tokenized_sentence}")  # 추가
    # 토큰화된 문장을 숫자 시퀀스로 변환
    sequence = tokenizer.texts_to_sequences([tokenized_sentence])
    print(f"시퀀스로 변환된 문장: {sequence}")  # 추가
    # 패딩 처리
    MAX_LENGTH = 100
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LENGTH)
    print(f"패딩 처리된 문장: {padded_sequence}")  # 추가
    # 예측 수행
    prediction = model.predict(padded_sequence)
    # 예측 확률 반환
    return prediction[0][0]

# 사용자로부터 문장 입력 받기
user_input = input("문장을 입력하세요: ")
predicted_prob = predict_label_prob(user_input)

print(f"라벨 0의 확률: {1 - predicted_prob:.4f}")
print(f"라벨 1의 확률: {predicted_prob:.4f}")


문장을 입력하세요: 통장 주민 번호 및 사칭 계좌 번호
토큰화된 문장: ['통장', '주민', '번호', '및', '사칭', '계좌', '번호']
시퀀스로 변환된 문장: [[12, 408, 50, 2356, 19, 50]]
패딩 처리된 문장: [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0   12  408   50 2356
    19   50]]
1/1 [==============================] - 1s 680ms/step
라벨 0의 확률: 0.1112
라벨 1의 확률: 0.8888


#### 2.3. SAVE/LOAD

2.3.1. save

In [ ]:
# 모델 저장
save_path_attention_lstm = '/content/drive/MyDrive/finalvoice/attention_lstm_model.h5'
model_attention_lstm.save(save_path_attention_lstm)

print(f"모델이 '{save_path_attention_lstm}' 경로에 저장되었습니다.")

모델이 '/content/drive/MyDrive/finalvoice/attention_lstm_model.h5' 경로에 저장되었습니다.


2.3.2. load

In [ ]:
from tensorflow.keras.models import load_model
save_path_attention_lstm = '/content/drive/MyDrive/finalvoice/attention_lstm_model.h5'
# 저장한 모델 불러오기
model_attention_lstm = load_model(save_path_attention_lstm, custom_objects={"AttentionLayer": AttentionLayer})

# 모델 요약 출력
model_attention_lstm.summary()

print(f"모델이 '{save_path_attention_lstm}' 경로에서 불러와졌습니다.")

NameError: ignored

#### 2.4. 성능 평가

In [ ]:
loss, accuracy = model_attention_lstm.evaluate(X_test_padded, y_test, batch_size=32)

print(f"Attention + LSTM 모델의 테스트 손실: {loss:.4f}")
print(f"Attention + LSTM 모델의 테스트 정확도: {accuracy:.4f}")

### 3.  KoBERT 모델

#### 3.1.모델 구축

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

# 학습률 조정
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
# 모델 구성 및 컴파일
model_kobert.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model_kobert.summary()

#### 3.2. 학습하기

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

In [ ]:
# KoBERT 토크나이저와 모델 로드
tokenizer_kobert = BertTokenizer.from_pretrained('monologg/kobert')
model_kobert = TFBertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2) # num_labels는 분류할 레이블의 수에 따라 조정하세요.

# 하이퍼파라미터 설정
MAX_LENGTH = 128  # 원하는 문장 최대 길이 설정

# KoBERT를 사용할 때, 입력 데이터는 토큰화 후 패딩 처리된 것이 아닌 원래의 텍스트 데이터를 사용해야 합니다.
train_encodings = tokenizer_kobert(list(X_train), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="tf")
test_encodings = tokenizer_kobert(list(X_test), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="tf")

# Early Stopping을 사용하여 성능 향상이 없을 때 학습을 조기 종료하는 기법을 적용하였습니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model_kobert.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# 모델 학습
history_kobert = model_kobert.fit(
    [train_encodings['input_ids'], train_encodings['attention_mask']], y_train,
    epochs=12,
    batch_size=32,
    validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']], y_test),
    callbacks=[early_stopping]
)

#### 3.3. SAVE/LOAD

save

In [ ]:
# 모델 저장하기
save_path = ('/content/drive/MyDrive/finalmodel/model_kobert')
model_kobert.save_pretrained(save_path)
tokenizer_kobert.save_pretrained(save_path)

print(f"모델과 토크나이저가 '{save_path}' 경로에 저장되었습니다.")

load

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# 저장한 모델과 토크나이저 불러오기
loaded_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')
loaded_model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')

print("모델과 토크나이저가 불러와졌습니다.")

모델 초기화 (필요 시)

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# 원본 모델과 토크나이저 로드
original_tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
original_model = TFBertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)

# 로드한 모델의 가중치를 원본 모델의 가중치로 복사
loaded_model.set_weights(original_model.get_weights())
loaded_tokenizer = original_tokenizer

print("모델과 토크나이저가 원본 모델에서 초기화되었습니다.")

#### 3.4. 평가하기

In [ ]:
from sklearn.metrics import accuracy_score

# 모델 컴파일
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 테스트 데이터의 인코딩
test_encodings = loaded_tokenizer(list(X_test), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="tf")

# 모델 성능 평가
loss, accuracy = model_kobert.evaluate(
    [test_encodings['input_ids'], test_encodings['attention_mask']], y_test
)

print(f"테스트 데이터에 대한 손실: {loss:.4f}")
print(f"테스트 데이터에 대한 정확도: {accuracy:.4f}")


### 4. 앙상블

#### 소프트 보팅 방법 채택

In [ ]:
from sklearn.metrics import accuracy_score, log_loss

# 모델 컴파일
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 테스트 데이터의 인코딩
test_encodings = loaded_tokenizer(list(X_test), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="tf")

# 각 모델의 예측 결과 계산
prob_kobert = loaded_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']]).logits
prob_attention_lstm = model_attention_lstm.predict(X_test_padded)

# 로짓 값을 확률로 변환
probabilities_kobert = tf.nn.sigmoid(prob_kobert)
probabilities_attention_lstm = tf.nn.sigmoid(prob_attention_lstm)

# 앙상블을 위한 예측 확률 계산
ensemble_probabilities = (probabilities_kobert + probabilities_attention_lstm) / 2

# 앙상블 결과를 소프트 보팅하여 예측 클래스 계산
ensemble_predicted_classes = np.argmax(ensemble_probabilities, axis=1)

# 정확도 평가
ensemble_accuracy = accuracy_score(y_test, ensemble_predicted_classes)
print(f"모델 앙상블의 예측 정확도: {ensemble_accuracy:.4f}")

# 로스 평가
ensemble_loss = log_loss(y_test, ensemble_probabilities)
print(f"모델 앙상블의 로스: {ensemble_loss:.4f}")

### 5. 최종 평가

#### 5.1. 모델 평가

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 정확도 평가
ensemble_accuracy = accuracy_score(y_test, ensemble_predicted_classes)
print(f"앙상블 모델의 예측 정확도: {ensemble_accuracy:.4f}")

# 정밀도 평가
precision = precision_score(y_test, ensemble_predicted_classes)
print(f"앙상블 모델의 정밀도: {precision:.4f}")

# 재현율 평가
recall = recall_score(y_test, ensemble_predicted_classes)
print(f"앙상블 모델의 재현율: {recall:.4f}")

# F1 점수 평가
f1 = f1_score(y_test, ensemble_predicted_classes)
print(f"앙상블 모델의 F1 점수: {f1:.4f}")

#### 5.2. I/O **테스트**

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np
import tensorflow as tf

# 불러온 모델 및 토크나이저 설정
loaded_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')
loaded_model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')

# 텍스트 입력 받기
text = input("예측할 텍스트를 입력하세요: ")

# 입력 텍스트의 전처리 및 인코딩
encoded_input = loaded_tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="tf")

# 각 모델의 예측 결과 계산
prob_kobert = loaded_model.predict([encoded_input['input_ids'], encoded_input['attention_mask']]).logits
prob_attention_lstm = model_attention_lstm.predict(X_test_padded)

# 로짓 값을 확률로 변환
probabilities_kobert = tf.nn.sigmoid(prob_kobert)
probabilities_attention_lstm = tf.nn.sigmoid(prob_attention_lstm)

# 앙상블을 위한 예측 확률 계산
ensemble_probabilities = (probabilities_kobert + probabilities_attention_lstm) / 2

# 앙상블 결과를 소프트 보팅하여 예측 클래스 계산
ensemble_predicted_class = np.argmax(ensemble_probabilities, axis=1)[0]

# 예측 클래스와 확률 출력
class_names = ['정상', '스팸']  # 클래스 이름 설정
predicted_class_name = class_names[ensemble_predicted_class]
predicted_class_probability = ensemble_probabilities[0][ensemble_predicted_class]
print(f"예측: {predicted_class_name}")
print(f"확률: {predicted_class_probability:.4f}")